# Imports


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Widgets

In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("storage_name", "adlsmartdatalavs2")
dbutils.widgets.text("container", "raw")
dbutils.widgets.text("catalogo", "chocolate_sales")
dbutils.widgets.text("esquema", "bronze")

# Parameters

In [0]:
storage_name = dbutils.widgets.get("storage_name")
container = dbutils.widgets.get("container")
catalogo = dbutils.widgets.get("catalogo")
esquema = dbutils.widgets.get("esquema")

# CSVs Path

In [0]:
route_salesman = f"abfss://{container}@{storage_name}.dfs.core.windows.net/master_salesman.csv"
route_country = f"abfss://{container}@{storage_name}.dfs.core.windows.net/master_country.csv"
route_product = f"abfss://{container}@{storage_name}.dfs.core.windows.net/master_product.csv"

# Schemas Structure

In [0]:
salesman_schema = StructType([
  StructField("salesman_name", StringType()),
  StructField("salesman_id", StringType())
  ])

country_schema = StructType([
  StructField("country", StringType()),
  StructField("country_id", StringType()),
  StructField("region", StringType()),
  StructField("region_id", StringType())
  ])

product_schema = StructType([
  StructField("product_desc", StringType()),
  StructField("product_id", StringType())
])

# Read Source


In [0]:
df_salesman = spark.read.option("header",True).schema(salesman_schema).csv(route_salesman)

df_salesman_final = df_salesman.withColumn("load_date", current_timestamp())
                                           
                                           
df_country = spark.read.option("header",True).schema(country_schema).csv(route_country)

df_country_final = df_country.withColumn("load_date", current_timestamp())          
                                           
                                           
df_product = spark.read.option("header",True).schema(product_schema).csv(route_product)

df_product_final = df_product.withColumn("load_date", current_timestamp())

# Save in table

In [0]:
df_salesman_final.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema}.master_salesman")

df_country_final.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema}.master_country")

df_product_final.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema}.master_product")